In [1]:
from tg.grammar_ru.ml.corpus import CorpusReader
from pathlib import Path
import os

base_path = Loc.corpus_path/'example.base.zip'))
reader = CorpusReader(Path(base_path))
db = reader.get_bundles().first()

In [30]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

cls = AntecedentCandidatesAlgorithm()
candidates_df = cls.get_candidates(db)
candidates_df

,pronoun_word_id,candidate_word_id,candidate_distance
0,10,0,0
1,18,0,1
2,18,10,0
3,30,0,2
4,30,10,1
...,...,...,...
428,921,907,4
429,921,914,3
430,921,917,2
431,921,918,1


In [31]:
with_parent_df = cls.get_pronoun_parent(db, candidates_df)
with_parent_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,10,0,0,9
1,18,0,1,19
2,18,10,0,19
3,30,0,2,31
4,30,10,1,31
...,...,...,...,...
428,921,907,4,922
429,921,914,3,922
430,921,917,2,922
431,921,918,1,922


In [107]:
from tg.grammar_ru.common import Loc
Loc.corpus_path

WindowsPath('c:/users/alexandra/desktop/grammar_ru/data-cache/corpus')

In [32]:
from tg.grammar_ru.common import Loc
from navec import Navec
import pandas as pd

navec = Navec.load(Loc.data_cache_path/'glove.tar')
words = list(navec.vocab.words)
ndf = pd.DataFrame(dict(word=words)).reset_index().set_index('word').rename(columns={'index':'navec_index'})
ndf

,navec_index
word,
a,0
a-а,1
aa,2
aaa,3
aan,4
...,...
ёь,499997
ёю,499998
ёё,499999


In [33]:
def set_navec_index(df, df_column, index_name):
    df = df.merge(ndf, left_on=df_column, right_index=True, how='left')
    df.navec_index = df.navec_index.fillna(-1).astype(int)
    df = df.rename(columns={'navec_index': index_name})
    return df

product_df = with_parent_df.merge(db.pymorphy.normal_form.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(db.pymorphy.normal_form.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
product_df = set_navec_index(product_df, 'pronoun_parent_norm', 'navec_index_norm_p')
product_df = set_navec_index(product_df, 'candidate_word_norm', 'navec_index_norm_c')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,navec_index_norm_p,navec_index_norm_c
0,10,0,0,9,преследовать,лилия,333934,196876
1,18,0,1,19,гнаться,лилия,91055,196876
2,18,10,0,19,гнаться,она,91055,267314
3,30,0,2,31,оставаться,лилия,273527,196876
4,30,10,1,31,оставаться,она,273527,267314
...,...,...,...,...,...,...,...,...
428,921,907,4,922,кубарем,жертва,187882,123688
429,921,914,3,922,кубарем,судьба,187882,424227
430,921,917,2,922,кубарем,она,187882,267314
431,921,918,1,922,кубарем,роль,187882,379575


In [34]:
pmph = db['pymorphy']
pronoun_form = pmph[pmph.index.isin(product_df['pronoun_word_id'])]['case']
product_df = product_df.merge(pronoun_form, left_on='pronoun_word_id', right_index=True, how='left')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,navec_index_norm_p,navec_index_norm_c,case
0,10,0,0,9,преследовать,лилия,333934,196876,accs
1,18,0,1,19,гнаться,лилия,91055,196876,nomn
2,18,10,0,19,гнаться,она,91055,267314,nomn
3,30,0,2,31,оставаться,лилия,273527,196876,datv
4,30,10,1,31,оставаться,она,273527,267314,datv
...,...,...,...,...,...,...,...,...,...
428,921,907,4,922,кубарем,жертва,187882,123688,nomn
429,921,914,3,922,кубарем,судьба,187882,424227,nomn
430,921,917,2,922,кубарем,она,187882,267314,nomn
431,921,918,1,922,кубарем,роль,187882,379575,nomn


In [35]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

product_df['candidate_inflected'] = product_df.apply(lambda x: inflect(x['candidate_word_norm'], x['case']), axis=1)
product_df = product_df.merge(db.src.word.rename('parent_original').str.lower(), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = set_navec_index(product_df, 'parent_original', 'navec_index_orig_p')
product_df = set_navec_index(product_df, 'candidate_inflected', 'navec_index_infl_c')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,navec_index_norm_p,navec_index_norm_c,case,candidate_inflected,parent_original,navec_index_orig_p,navec_index_infl_c
0,10,0,0,9,преследовать,лилия,333934,196876,accs,лилию,преследовал,333907,196875
1,18,0,1,19,гнаться,лилия,91055,196876,nomn,лилия,гналась,91045,196876
2,18,10,0,19,гнаться,она,91055,267314,nomn,она,гналась,91045,267314
3,30,0,2,31,оставаться,лилия,273527,196876,datv,лилии,оставался,273526,196858
4,30,10,1,31,оставаться,она,273527,267314,datv,ей,оставался,273526,121060
...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,921,907,4,922,кубарем,жертва,187882,123688,nomn,жертва,кубарем,187882,123688
429,921,914,3,922,кубарем,судьба,187882,424227,nomn,судьба,кубарем,187882,424227
430,921,917,2,922,кубарем,она,187882,267314,nomn,она,кубарем,187882,267314
431,921,918,1,922,кубарем,роль,187882,379575,nomn,роль,кубарем,187882,379575


In [36]:
from slovnet.model.emb import NavecEmbedding
import torch

emb = NavecEmbedding(navec)
navec_indices = pd.concat([product_df.navec_index_norm_p, 
                           product_df.navec_index_norm_c,
                           product_df.navec_index_orig_p,
                           product_df.navec_index_infl_c]).unique()
input = torch.tensor([x for x in navec_indices if x!=-1])
output = emb(input.long())

gdf = pd.DataFrame(output.tolist())
gdf['navec_index'] = input.tolist()
gdf = gdf.set_index('navec_index')
gdf

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
navec_index,,,,,,,,,,,,,,,,,,,,,
333934,-0.112453,-0.306758,0.145285,0.443198,0.012505,-0.402717,0.076367,-0.220459,-0.387291,-0.110226,...,-0.016714,-0.334920,-0.647631,-0.436893,-0.075502,0.024846,-0.085273,-0.563138,-0.509545,0.423477
91055,0.197562,0.080616,0.202884,-0.145310,-0.210084,-0.233264,-0.366435,-0.153203,0.040027,-0.130637,...,-0.285696,-0.110329,-0.043071,-0.677648,0.052566,0.173356,0.185777,-0.563138,-0.509545,0.423477
273527,0.324312,-0.407607,0.037182,0.258735,0.011883,-0.093407,-0.272309,-0.170452,-0.346471,0.389134,...,0.608405,-0.013813,-0.186715,-0.477247,0.238254,-0.013511,0.056895,-0.169360,-0.220133,0.849850
279254,-0.114882,-0.145657,-0.000606,-0.003198,-0.379298,0.049046,0.040712,-0.146096,-0.085722,0.157008,...,0.358176,0.438139,-0.273677,-0.452074,0.196641,-0.409728,0.254586,-0.221693,0.168994,-0.096643
306432,-0.197706,0.084281,0.263474,-0.377358,-0.277203,-0.009678,0.225207,-0.082434,0.381535,-0.428462,...,0.394494,-0.251483,-0.158747,-0.058563,0.333999,0.179716,-0.109330,-0.062578,0.218991,0.083121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356006,0.031675,0.324081,-0.347707,0.181894,-0.230779,0.424414,-0.278897,0.194534,0.065564,-0.109867,...,0.248024,0.149218,0.158946,0.181778,-0.112056,0.058064,0.227423,0.733654,0.369591,0.111176
430919,0.021681,0.125735,0.251885,0.291416,0.111534,0.172244,-0.100932,0.270803,-0.265560,-0.142112,...,-0.580810,0.044219,0.162369,0.055797,0.235268,0.054568,0.218531,0.138013,0.243070,0.168594
286159,-0.020386,-0.430457,-0.312975,0.356807,-0.051699,-0.759013,0.193865,0.405029,-0.045148,-0.282417,...,-0.425034,0.535372,-0.295388,-0.221495,-0.469748,-0.408250,-0.391878,-0.080437,-0.233745,-0.014317


In [37]:
from tg.common.ml.miscellaneous.glove import GloveProcessor

def substract_embeddings_square(x_index, y_index):
    x = x_index.to_frame('idx').merge(gdf, left_on='idx', right_index=True, how='left').drop('idx', axis=1)
    y = y_index.to_frame('idx').merge(gdf, left_on='idx', right_index=True, how='left').drop('idx', axis=1)
    sub = x - y
    return (sub * sub).sum(axis=1)

product_df['norm_glove_square'] = substract_embeddings_square(product_df.navec_index_norm_p, product_df.navec_index_norm_c)
product_df['inflected_glove_square'] = substract_embeddings_square(product_df.navec_index_orig_p, product_df.navec_index_infl_c)
product_df[['pronoun_word_id', 'candidate_word_id', 'candidate_inflected', 'parent_original', 'norm_glove_square', 'inflected_glove_square']].head(40)

,pronoun_word_id,candidate_word_id,candidate_inflected,parent_original,norm_glove_square,inflected_glove_square
0,10,0,лилию,преследовал,60.801662,55.235951
1,18,0,лилия,гналась,55.151395,56.689454
2,18,10,она,гналась,55.543009,57.023314
3,30,0,лилии,оставался,60.209949,72.378968
4,30,10,ей,оставался,41.402170,47.968986
5,30,18,ей,оставался,41.402170,47.968986
6,48,3,суетливого,оторваться,64.350067,60.892517
7,48,5,путаного,оторваться,71.380800,69.869130
8,48,6,сна,оторваться,58.981638,55.766872
9,48,8,кого-то,оторваться,55.649469,51.369442


In [38]:
max_norm_square = product_df.groupby(['pronoun_word_id'])['norm_glove_square'].idxmax()
max_norm_square = max_norm_square.dropna().astype(int)

In [13]:
on_norm = product_df.loc[max_norm_square][['pronoun_word_id', 'candidate_word_id', 'candidate_distance', 'pronoun_parent_norm', 'candidate_word_norm', 'norm_glove_square']]
on_norm.head(10)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_norm,candidate_word_norm,norm_glove_square
0,10,0,0,преследовать,лилия,60.801662
2,18,10,0,гнаться,она,55.543009
3,30,0,2,оставаться,лилия,60.209949
7,48,5,6,оторваться,путаный,71.380800
14,53,0,3,подвести,лилия,56.628594
18,56,0,4,упасть,лилия,59.064480
32,83,82,0,увидеть,спальня,56.931019
40,113,105,2,напугать,кирпичный,78.826817
48,122,105,4,чувствовать,кирпичный,84.068237
55,145,106,7,подкараулить,стена,77.216060


Кто-то выше занялся ерундой.

In [39]:
from tg.common.ml.miscellaneous.glove import GloveProcessor

product_df['product_norm'] = GloveProcessor.apply_scores(product_df.navec_index_norm_p, product_df.navec_index_norm_c, gdf)
product_df['product_infl'] = GloveProcessor.apply_scores(product_df.navec_index_orig_p, product_df.navec_index_infl_c, gdf)
max_prod = product_df.groupby(['pronoun_word_id'])['product_norm'].idxmax()
max_prod = max_prod.dropna().astype(int)
product_df.loc[max_prod][['pronoun_word_id', 'candidate_word_id', 'candidate_distance', 'pronoun_parent_norm', 'candidate_word_norm', 'product_norm']]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_norm,candidate_word_norm,product_norm
0,10,0,0,преследовать,лилия,-2.059554
2,18,10,0,гнаться,она,-0.213960
4,30,10,1,оставаться,она,8.539292
8,48,6,5,оторваться,сон,5.280543
15,53,10,2,подвести,она,1.963640
19,56,10,3,упасть,она,7.820927
24,83,10,8,увидеть,она,10.808409
42,113,110,0,напугать,ворона,4.477386
51,122,113,1,чувствовать,она,10.140821
56,145,110,6,подкараулить,ворона,0.797098


In [40]:
max_prod = product_df.groupby(['pronoun_word_id'])['product_infl'].idxmax()
max_prod = max_prod.dropna().astype(int)
product_df.loc[max_prod][['pronoun_word_id', 'candidate_word_id', 'candidate_distance', 'pronoun_parent_norm', 'candidate_word_norm', 'product_norm']]

,pronoun_word_id,candidate_word_id,candidate_distance,candidate_inflected,parent_original,inflected_glove_square
0,10,0,0,лилию,преследовал,55.235951
2,18,10,0,она,гналась,57.023314
3,30,0,2,лилии,оставался,72.378968
7,48,5,6,путаного,оторваться,69.869130
14,53,0,3,лилию,подвели,55.526743
18,56,0,4,лилия,упала,45.886245
29,83,61,3,пустота,увидела,61.751094
40,113,105,2,кирпичный,напугала,71.787097
48,122,105,4,кирпичный,чувствовала,83.369851
60,145,124,2,тревогу,подкараулили,79.414895


Ошибки:

0) Лилия очнулась от суетливого, путаного сна. Кто-то преследовал ее

8) путаного сна ... схватить _злодея_ — или, возможно, чтобы окончательно оторваться от него

42) Недалеко каркали вороны, которых она напугала

56) каркали вороны ... Лилия вспомнила. Ее подкараулили!

70) тревогу, расходившуюся волнами ... Лилия вспомнила. Ее подкараулили!

85) Голова кружилась ...  мир в ее глазах замерцал

128) Кто-нибудь другой ... С простым грабителем она бы в два счета разделалась

142) порча, поразившая споенец ... Ее хотел убить темный колдун

...

В фильтрации надо было использовать slovnet, а не pymorphy

In [2]:
# фильтр на slovnet и меньше кандидатов на местоимение
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

cls = AntecedentCandidatesAlgorithm(7)
candidates_df = cls.get_candidates(db)
with_parent_df = cls.get_pronoun_parent(db, candidates_df)
with_parent_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,10,0,0,9
1,18,0,1,19
2,18,10,0,19
3,30,0,2,31
4,30,10,1,31
...,...,...,...,...
303,921,907,4,922
304,921,914,3,922
305,921,917,2,922
306,921,918,1,922


In [5]:
from tg.grammar_ru.ml.features import NavecFeaturizer

prod_feat = NavecFeaturizer()

product_df = with_parent_df.merge(db.pymorphy.normal_form.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(db.pymorphy.normal_form.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
product_df['norm_prod'] = prod_feat.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,norm_prod
0,10,0,0,9,преследовать,лилия,-2.059554
1,18,0,1,19,гнаться,лилия,-1.109730
2,18,10,0,19,гнаться,она,-0.213960
3,30,0,2,31,оставаться,лилия,-1.956175
4,30,10,1,31,оставаться,она,8.539292
...,...,...,...,...,...,...,...
303,921,907,4,922,кубарем,жертва,-0.980053
304,921,914,3,922,кубарем,судьба,-1.951545
305,921,917,2,922,кубарем,она,-0.423255
306,921,918,1,922,кубарем,роль,-1.972080


In [7]:
def get_max(df, group_col, max_col):
    max_rows = df.groupby([group_col])[max_col].idxmax()
    max_rows = max_rows.dropna().astype(int)
    return df.loc[max_rows]

get_max(product_df, 'pronoun_word_id', 'norm_prod')

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,norm_prod
0,10,0,0,9,преследовать,лилия,-2.059554
2,18,10,0,19,гнаться,она,-0.213960
4,30,10,1,31,оставаться,она,8.539292
8,48,6,3,46,оторваться,сон,5.280543
13,53,10,2,52,подвести,она,1.963640
17,56,10,3,57,упасть,она,7.820927
21,83,30,6,84,увидеть,она,10.808409
30,113,95,4,114,напугать,девушка,5.793210
40,122,113,1,123,чувствовать,она,10.140821
48,145,142,0,146,подкараулить,лилия,-0.378904


Замены имён собственных:

In [30]:
words = db.pymorphy[db.pymorphy['POS'] == 'NOUN']
head = words.groupby(['normal_form'])['normal_form'].count().sort_values(ascending=False).head(10)
top_words = head.to_frame().rename(columns={'normal_form': 'count'}).reset_index()
top_words

,normal_form,count
0,лилия,19
1,глаз,5
2,споенец,4
3,колдун,4
4,тревога,3
5,ворота,3
6,заклинание,3
7,дом,3
8,тело,3
9,грабитель,3


In [58]:
def is_noun_proper(noun):
    indices = list(db.pymorphy[db.pymorphy['normal_form'] == noun].index)
    words = db.src[db.src.index.isin(indices)]['word']
    return words.str[0].str.isupper().all()

top_words['is_proper'] = top_words['normal_form'].map(is_noun_proper)
top_words

,normal_form,count,is_proper
0,лилия,19,True
1,глаз,5,False
2,споенец,4,False
3,колдун,4,False
4,тревога,3,False
5,ворота,3,False
6,заклинание,3,False
7,дом,3,False
8,тело,3,False
9,грабитель,3,False


In [86]:
propers = list(top_words[top_words['is_proper']]['normal_form'])
replace = {'masc': 'мужчина', 'femn': 'женщина', 'neut': 'существо'}
replaced = list(map(lambda x: replace[db.pymorphy[db.pymorphy.normal_form == x].iloc[0]['gender']], propers))
propers_replace = dict(zip(propers, replaced))
propers_replace

{'лилия': 'женщина'}

In [99]:
replaces_series = db.pymorphy['normal_form'].apply(lambda x: propers_replace[x] if x in propers else x)
replaces_series

word_id
0         женщина
1        очнуться
2              от
3       суетливый
4               ,
          ...    
921           она
922       кубарем
923    покатиться
924          вниз
925             .
Name: normal_form, Length: 926, dtype: object

In [102]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm
from tg.grammar_ru.ml.features import NavecFeaturizer

cls = AntecedentCandidatesAlgorithm(7)
prod_feat = NavecFeaturizer()

with_parent_df = cls.get_pronoun_parent(db)
product_df = with_parent_df.merge(replaces_series.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(replaces_series.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
product_df['norm_prod'] = prod_feat.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
get_max(product_df, 'pronoun_word_id', 'norm_prod')

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,norm_prod
0,10,0,0,9,преследовать,женщина,2.660135
2,18,10,0,19,гнаться,она,-0.213960
4,30,10,1,31,оставаться,она,8.539292
8,48,6,3,46,оторваться,сон,5.280543
13,53,10,2,52,подвести,она,1.963640
17,56,10,3,57,упасть,она,7.820927
21,83,30,6,84,увидеть,она,10.808409
30,113,95,4,114,напугать,девушка,5.793210
40,122,113,1,123,чувствовать,она,10.140821
42,145,105,6,146,подкараулить,кирпичный,-0.722384


In [103]:
cls = AntecedentCandidatesAlgorithm(5)
with_parent_df = cls.get_pronoun_parent(db)
product_df = with_parent_df.merge(replaces_series.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(replaces_series.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
product_df['norm_prod'] = prod_feat.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
get_max(product_df, 'pronoun_word_id', 'norm_prod')

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,norm_prod
0,10,0,0,9,преследовать,женщина,2.660135
2,18,10,0,19,гнаться,она,-0.213960
4,30,10,1,31,оставаться,она,8.539292
7,48,6,3,46,оторваться,сон,5.280543
12,53,10,2,52,подвести,она,1.963640
16,56,10,3,57,упасть,она,7.820927
20,83,56,4,84,увидеть,она,10.808409
25,113,95,4,114,напугать,девушка,5.793210
33,122,113,1,123,чувствовать,она,10.140821
38,145,124,1,146,подкараулить,тревога,-2.088039
